In [116]:
import re
import camelot
import pickle
import pandas as pd

pickle_file_path = "../output/caljobs_activity_codes_detailed_listing_camelot_output.pkl"

tbls = camelot.read_pdf("../sourcedata/caljobs_activity_codes_detailed_listing.pdf", pages="1-9")

with open(pickle_file_path, "wb") as handle:
    pickle.dump(tbls, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [118]:
with open(pickle_file_path, "rb") as handle:
    tbls_pkl = pickle.load(handle)

activity_code = []
programs_list = []
reporting_category = []
restarts_exit_clock = []
pirl = []
duration = []

for tbl in tbls_pkl:
    df = tbl.df.tail(-2)
    activity_code.extend(df[0])
    programs_list.append(df[list(range(2,8))].apply(lambda x: x.str.strip().replace("", "0").replace("x", "1")))
    reporting_category.extend(df[8].replace("\n", "", regex=True))
    restarts_exit_clock.extend(df[9].str.strip().replace("NO", "0").replace("YES", "1"))
    pirl.extend(df[10].replace("\n", "", regex=True))
    duration.extend(df[11].str.strip())

programs_df = pd.concat(programs_list, ignore_index=True)
programs_df.columns = ["prog_affiliation_adult_dw",
           "prog_affiliation_youth",
           "prog_affiliation_nfjp",
           "prog_affiliation_wp_jvsg",
           "prog_affiliation_msfw",
           "prog_affiliation_taa"]

caljobs_act_codes_detailed_list = pd.concat([
    pd.Series(activity_code).rename("activity_code"),
    programs_df,
    pd.Series(reporting_category).rename("reporting_category"),
    pd.Series(restarts_exit_clock).rename("restarts_exit_clock"),
    pd.Series(pirl).rename("pirl"),
    pd.Series(duration).rename("duration")
], axis=1)

caljobs_act_codes_detailed_list.loc[4,"activity_code"] = "06M"

caljobs_act_codes_detailed_list.to_csv("../output/caljobs_activity_codes_detailed_listing.csv")